<a href="https://colab.research.google.com/github/Kashara-Alvin-Ssali/Machine-Learning/blob/main/TRestNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow keras numpy matplotlib


In [ ]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from google.colab import files

In [ ]:
from google.colab import drive  # <-- Import drive
drive.mount('/content/drive')

# Define path to Dataset.zip in Google Drive
zip_path = "/content/drive/My Drive/Dataset.zip"
extract_path = "./Dataset"

# Extract Dataset.zip
import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted successfully!


In [ ]:
train_dir = "./Dataset/Dataset/Training"
validation_dir = "./Dataset/Dataset/Validation"
test_dir = "./Dataset/Dataset/Testing"  # If needed for testing


In [ ]:
# Image dimensions
height = 300
width = 300
batch_size = 8


In [ ]:
# Load ResNet50 without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(height, width, 3))

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=True
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Added test data generator

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)
validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 62 images belonging to 2 classes.
Found 18 images belonging to 2 classes.
Found 15 images belonging to 2 classes.


In [ ]:
# Add custom layers to ResNet50
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Callbacks
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
# Train the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.6691 - loss: 32.2141

8/8 ━━━━━━━━━━━━━━━━━━━━ 137s 13s/step - accuracy: 0.6664 - loss: 33.9962 - val_accuracy: 0.6667 - val_loss: 362.6860
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 81s 10s/step - accuracy: 0.4535 - loss: 24.0844 - val_accuracy: 0.5000 - val_loss: 683232.2500
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 72s 9s/step - accuracy: 0.4886 - loss: 14.6260 - val_accuracy: 0.4444 - val_loss: 10735441920.0000
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 63s 8s/step - accuracy: 0.5094 - loss: 37.6874 - val_accuracy: 0.4444 - val_loss: 1000722688.0000
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 82s 8s/step - accuracy: 0.5971 - loss: 3.1509 - val_accuracy: 0.4444 - val_loss: 98631688.0000
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 64s 8s/step - accuracy: 0.5938 - loss: 5.3679 - val_accuracy: 0.4444 - val_loss: 1488636.2500


In [ ]:
# Save the final model
model.save("resnet50_model.h5")
print("Model training complete and saved as resnet50_model.h5")

Model training complete and saved as resnet50_model.h5


In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2%}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.6056 - loss: 2552.3850
Test Accuracy: 53.33%


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img_path = "/content/Dataset/Dataset/Testing/Fake/500_f1.jpg"  # Corrected path
img = image.load_img(img_path, target_size=(300, 300))  # Match model input size
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = preprocess_input(img_array)  # Apply same preprocessing as training

# Make a prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)  # Get class index
class_labels = list(train_generator.class_indices.keys())  # Get class names
print(f"Predicted Class: {class_labels[predicted_class]}")  # Print the class label

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Class: Real
